In [1]:
using LinearAlgebra

"""
    resolver_tridiagonal_crout(l, d, u, b)

Resuelve un sistema de ecuaciones lineales tridiagonal Ax = b utilizando el
algoritmo de Crout optimizado.

# Argumentos
- `l::Vector{Float64}`: Vector de la subdiagonal (longitud n-1).
- `d::Vector{Float64}`: Vector de la diagonal principal (longitud n).
- `u::Vector{Float64}`: Vector de la superdiagonal (longitud n-1).
- `b::Vector{Float64}`: El vector del lado derecho del sistema (longitud n).

# Retorna
- `Vector{Float64}`: El vector de solución `x`.
"""
function resolver_tridiagonal_crout(l::Vector{Float64}, d::Vector{Float64}, u::Vector{Float64}, b::Vector{Float64})
    n = length(d)
    if !(length(l) == n-1 && length(u) == n-1 && length(b) == n)
        error("Las dimensiones de los vectores no son correctas para un sistema de n x n.")
    end

    # Vectores para almacenar los elementos calculados de L, U y z
    l_diag = zeros(n)
    u_super = zeros(n - 1)
    z = zeros(n)
    
    println("--- Iniciando Factorización de Crout y Sustitución Adelante ---")
    
    # --- Pasos 1-3: Configuran y resuelven Lz = b ---
    
    # Paso 1: Inicialización para i = 1
    l_diag[1] = d[1]
    u_super[1] = u[1] / l_diag[1]
    z[1] = b[1] / l_diag[1]
    
    # Paso 2: Bucle para i = 2, ..., n-1
    for i in 2:(n-1)
        # El subdiagonal de L es el mismo que el de A (lᵢ,ᵢ₋₁ = aᵢ,ᵢ₋₁)
        l_diag[i] = d[i] - l[i-1] * u_super[i-1]
        u_super[i] = u[i] / l_diag[i]
        z[i] = (b[i] - l[i-1] * z[i-1]) / l_diag[i]
    end
    
    # Paso 3: Cálculo final para i = n
    l_diag[n] = d[n] - l[n-1] * u_super[n-1]
    z[n] = (b[n] - l[n-1] * z[n-1]) / l_diag[n]
    
    println("Factorización y sustitución adelante completadas.")
    println("Vector intermedio z: ", round.(z, digits=4))
    
    # --- Pasos 4-5: Resuelven Ux = z ---
    
    println("\n--- Iniciando Sustitución Hacia Atrás ---")
    x = zeros(n)
    
    # Paso 4: Determinar xₙ
    x[n] = z[n]
    
    # Paso 5: Bucle hacia atrás para i = n-1, ..., 1
    for i in (n-1):-1:1
        x[i] = z[i] - u_super[i] * x[i+1]
    end
    
    # Paso 6: SALIDA
    println("Sustitución hacia atrás completada.")
    return x
end

resolver_tridiagonal_crout

In [2]:
# Sistema de ecuaciones a resolver:
# 2x₁ -  x₂          = 1
# -x₁ + 2x₂ -  x₃    = 0
#      - x₂ + 2x₃ - x₄ = 0
#           -  x₃ + 2x₄ = 1

# Definimos la matriz por sus diagonales
d = [2.0, 2.0, 2.0, 2.0] # Diagonal principal
u = [-1.0, -1.0, -1.0]   # Superdiagonal (arriba)
l = [-1.0, -1.0, -1.0]   # Subdiagonal (abajo)
b = [1.0, 0.0, 0.0, 1.0]   # Vector de resultados

# Llamamos a la función para resolver el sistema
solucion_x = resolver_tridiagonal_crout(l, d, u, b)

println("\n--- RESULTADO FINAL ---")
println("La solución del sistema es x = ")
display(round.(solucion_x, digits=4))

# --- Verificación ---
# Reconstruimos la matriz A completa para verificar el resultado
A_completa = Tridiagonal(l, d, u)
println("\n--- Verificación: A * x ---")
b_calculado = A_completa * solucion_x
println("El resultado de A*x es (debería ser igual a b):")
display(round.(b_calculado, digits=4))

if isapprox(b, b_calculado)
    println("\n✅ Verificación exitosa: A*x es igual a b.")
else
    println("\n❌ Error en la verificación.")
end

--- Iniciando Factorización de Crout y Sustitución Adelante ---
Factorización y sustitución adelante completadas.
Vector intermedio z: [0.5, 0.3333, 0.25, 1.0]

--- Iniciando Sustitución Hacia Atrás ---
Sustitución hacia atrás completada.

--- RESULTADO FINAL ---
La solución del sistema es x = 


4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0


--- Verificación: A * x ---
El resultado de A*x es (debería ser igual a b):


4-element Vector{Float64}:
  1.0
 -0.0
  0.0
  1.0


✅ Verificación exitosa: A*x es igual a b.
